# Module
## Model I/O
### 언어 모델과의 Interface
- Prompts
- Language models
- Output parsers

## Retrieval
### 외부 데이터로 접근하여 이를 모델에 어떻게 제공하는 것
- document loaders
- Transformers
- Text embedding models
- Vector sotres
- retrievers

## Chains

## Agent
### AI가 독립적으로 작동하도록 만들 수 있게 해주는 역할

## Memory
### 챗봇에 기억할 수 있도록 하는 역할

## Callbacks
### model이 기본적으로 무엇을 하고 있는지 중간에 확인하는 역할



In [1]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotPromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts.prompt import PromptTemplate


## 4.1
#### FewShotPromptTemplate

In [2]:
chat = ChatOpenAI(
    temperature = 0.1,
    streaming = True,
    callbacks = [
        StreamingStdOutCallbackHandler(),
    ],
)

In [3]:
examples = [
    {
        "question": "What do you know about France?",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Italy?",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "question": "What do you know about Greece?",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [5]:
example_prompt = PromptTemplate.from_template("Human: {question}\nAI: {answer}")

prompt = FewShotPromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
    suffix = "Humana: What do you know about {country}?",
    input_variables = ["country"],
)

chain = prompt | chat
chain.invoke({"country": "Korean"})

AI: 
        Here is what I know:
        Capital: Seoul
        Language: Korean
        Food: Kimchi and Bibimbap
        Currency: South Korean Won

AIMessage(content='AI: \n        Here is what I know:\n        Capital: Seoul\n        Language: Korean\n        Food: Kimchi and Bibimbap\n        Currency: South Korean Won', response_metadata={'finish_reason': 'stop'}, id='run-e2f6683b-b3d0-4695-8181-dd131467d020-0')

## 4.2
#### FewShotChatMessagePromptTemplate

In [6]:
from langchain.chat_models import ChatOpenAI
from langchain.prompts.few_shot import FewShotChatMessagePromptTemplate
from langchain.callbacks import StreamingStdOutCallbackHandler
from langchain.prompts import ChatMessagePromptTemplate, ChatPromptTemplate

In [7]:
chat = ChatOpenAI(
    temperature=0.1,
    streaming=True,
    callbacks=[
        StreamingStdOutCallbackHandler(),
    ],
)


examples = [
    {
        "country": "France",
        "answer": """
        Here is what I know:
        Capital: Paris
        Language: French
        Food: Wine and Cheese
        Currency: Euro
        """,
    },
    {
        "country": "Italy",
        "answer": """
        I know this:
        Capital: Rome
        Language: Italian
        Food: Pizza and Pasta
        Currency: Euro
        """,
    },
    {
        "country": "Greece",
        "answer": """
        I know this:
        Capital: Athens
        Language: Greek
        Food: Souvlaki and Feta Cheese
        Currency: Euro
        """,
    },
]

In [9]:
example_prompt = ChatPromptTemplate.from_messages(
    [
        ("human", "What do you know about {country}?"),
        ("ai", "{answer}"),
    ]
)

example_prompt = FewShotChatMessagePromptTemplate(
    example_prompt = example_prompt,
    examples = examples,
)

In [11]:
final_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a geography expert, you give short answers."),
        example_prompt,
        ("human", "What do you know about {country}?"),
    ]
)

chain = final_prompt | chat
chain.invoke({"country": "Korean"})

I assume you are referring to South Korea. Here is what I know:
Capital: Seoul
Language: Korean
Food: Kimchi and Bibimbap
Currency: South Korean Won

AIMessage(content='I assume you are referring to South Korea. Here is what I know:\nCapital: Seoul\nLanguage: Korean\nFood: Kimchi and Bibimbap\nCurrency: South Korean Won', response_metadata={'finish_reason': 'stop'}, id='run-7ebfa42b-209d-4856-a0e8-533c3c4f2c5f-0')